In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Cho dữ liệu automobileEDA.csv, xây dựng model dự đoán price của xe hơi dựa trên highway-mpg (miles-per-gallon)

In [ ]:
df = pd.read_csv('data/automobileEDA.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
sns.regplot(data=df, x='highway-mpg', y='price'); # có tương quan nghịch (trong từng phân khúc)

In [ ]:
df[['highway-mpg','price']].corr() # tương quan khá mạnh và nghịch

In [ ]:
sns.residplot(data=df, x='highway-mpg', y='price'); # sai số y-y^
# Chúng ta có thể thấy từ residual plot này như sau: phần dư khá cao và không được trải đều trên trục x, 
# điều này khiến chúng ta tin rằng có thể một mô hình phi tuyến tính (non-linear model) phù hợp hơn với dữ liệu này.

In [ ]:
df[['highway-mpg','price']].describe() # phương sai biến price khá lớn

In [ ]:
df['price'].skew() # lệch phải

In [ ]:
sns.displot(data=df, x='highway-mpg', kde=True)
plt.show()
# xấp xỉ phân phối chuẩn

In [ ]:
sns.boxplot(data=df, x='highway-mpg')
plt.show()
# có ít ouliers

In [ ]:
sns.displot(data=df, x='price', kde=True)
plt.show()
# phân phối lệch phải

In [ ]:
sns.boxplot(data=df, x='price')
plt.show()
# có nhiều outliers

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
X = df[['highway-mpg']] # X = df['highway-mpg'] --> SAI
y = df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2) # mặc định test_size là 0.25

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
# y = bx + a
# y = b1x1 + b2x2 + b3*x3 + a

In [ ]:
yhat = lm.predict(X_test)
# y_test: thực tế
# yhat: dự đoán
# y - y^

In [ ]:
yhat[:5]

In [ ]:
y_test[:5]

In [ ]:
a = lm.intercept_
b = lm.coef_[0]
a, b

In [ ]:
print('Price =', b,'x hightway-mpg +', a) # y = bx + a

In [ ]:
print('The full R-square is:', lm.score(X,y))
# Với kết quả trên có thể nói rằng chỉ ~ 48% variation của price được giải thích bằng simple linear này. 

In [ ]:
print('The train R-square is:', lm.score(X_train,y_train))
print('The test R-square is:', lm.score(X_test,y_test))
# Như vậy, các biến độc lập giải thích được 42.5% sự biến thiên của biến phụ thuộc. 
# Phần còn lại 57.5% được giải thích bởi các biến ngoài mô hình và sai số ngẫu nhiên.

# Kèm theo đó, kết quả so sánh giữa R^2 của cả train và test đều thấp.
# => Model này chưa phù hợp với dữ liệu.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print('The train MSE:', mean_squared_error(y_train, lm.predict(X_train)))
print('The test MSE:', mean_squared_error(y_test, yhat))

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
print('The train MAE:', mean_absolute_error(y_train, lm.predict(X_train)))
print('The test MAE:', mean_absolute_error(y_test, yhat))

In [ ]:
# MSE, MAE cao cho thấy sự chênh lệch lớn giữa y dự đoán và y thực tế

## So sánh giá xe dự đoán (yhat) và giá xe thực tế (y_test) của tập test

In [ ]:
plt.scatter(yhat, y_test)
plt.xlabel('Model Predictions')
plt.ylabel('True Value')
plt.plot([0, 50000], [0, 50000], '-', color = 'r') # y=x; mong muốn y và y^ bằng nhau
plt.title('Biểu đồ so sánh giá xe dự đoán và thực tế của test')
plt.show()

- Đặt giới hạn trục x và y là 50000 để chúng ta có được cái nhìn rõ hơn về vị trí của hầu hết các điểm dữ liệu. 
- Thêm line có độ dốc 1 đóng vai trò là tham chiếu. Nếu tất cả các điểm nằm trên line này, điều đó có nghĩa là có một mối quan hệ hoàn hảo giữa thực tế  và dự đoán.

## So sánh giá xe dự đoán và giá xe thực tế của hai tập train và test 

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
ax1 = sns.distplot(y_train, hist=False, color="r", label="Actual Train")
sns.distplot(lm.predict(X_train), hist=False, color="b", label="Predicted Train", ax=ax1)
plt.legend()

plt.subplot(1,2,2)
ax2 = sns.distplot(y_test, hist=False, color="r", label="Actual Test")
sns.distplot(lm.predict(X_test), hist=False, color="b", label="Predicted Test" , ax=ax2)
plt.legend()

plt.tight_layout()
plt.show()
# tập test dự đoán có sự khác biệt nhiều hơn tập train

## Feature Engineering

In [ ]:
# Thử Standard scaler cho biến highway-mpg
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[['highway-mpg']])

In [ ]:
df['highway-mpg-scale'] = scaled_df

In [ ]:
df[['highway-mpg','highway-mpg-scale']].head()

In [ ]:
# Thử Log normalization biến price
df['price_log'] = np.log(df['price'])

In [ ]:
df[['highway-mpg','highway-mpg-scale','price','price_log']].head()

In [ ]:
X = df[['highway-mpg-scale']]
y = df['price_log']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

In [ ]:
print('The full R-square is:', lm.score(X,y))
print('The train R-square is:', lm.score(X_train,y_train))
print('The test R-square is:', lm.score(X_test,y_test))
# các score đều tăng nhưng không đáng kể ==> cần thêm các feature